### Methods proposed in the article 

In this notebook we will implement in ``Python`` using ``Numpy`` the algorithms proposed in the paper "Dimension Independent Matrix Square using
MapReduce (DIMSUM)"

We remember that $A$ is a sparse matrix of size $m \times n$ with $L$ nonzero elements per row, and our goal is to compute $A^T A$

In [1]:
import numpy as np
import os
import sys
project_dir = os.getcwd().split('notebooks')[0]
sys.path.append(project_dir)

In [2]:
from src.helper import *

#### 1) Algorithm 1: NaiveMapper

In [3]:
def NaiveMapper(ri):
    """
    Mapper function to naive matrix multiplication
    input: 
    ri: arrays of a row i of the sparse matrix A
    output:
    AProd: matrix with dot products between pairs of the row ri
    """
    AProd =  [ri[i]*ri for i in range(len(ri))]  # we walk through the rows

    return np.matrix(AProd)

#### 2) Algorithm 2: NaiveReducer

In [4]:
def NaiveReducer(col_maps):
    """
    Reducer function to naive matrix multiplication
    input:
    col_maps: list of mappings
    output:
    result_mat = matrix with the sum of mappings
    """
    m = len(col_maps)
    (nrow, ncol) = col_maps[0].shape
    result_mat = np.zeros((nrow, ncol) )
    for lm in col_maps:
        result_mat+=np.matrix(lm)
    return result_mat

In [5]:
def NaiveProd(A):
    """
    Function that computes A^T A in a naive way
    input:
    A: sparse matrix
    output: matrix product A^T A
    """
    (m,n) = A.shape
    rows = [A[i] for i in range(m) ]
    col_maps = [NaiveMapper(rows[j]) for j in range(m)]
    return NaiveReducer(col_maps)

In [6]:
A = sparse_generator(4,3)

In [7]:
NaiveProd(A)

array([[0.07873407, 0.        , 0.        ],
       [0.        , 0.62951415, 0.        ],
       [0.        , 0.        , 0.59050071]])

Verifying our result with the ``Numpy`` function that computes matrix product using ``A.T@A``:

In [8]:
A.T@A

array([[0.07873407, 0.        , 0.        ],
       [0.        , 0.62951415, 0.        ],
       [0.        , 0.        , 0.59050071]])

#### 3) Algorithm 3: DIMSUMMapper

In [9]:
def norm(A):
    """
    Function that computes the norm 2 of each column of a matrix
    input:
    A: matrix
    output:
    norm: array of norms of each column
    """
    norm = (np.square(A).sum(axis=0))**(1/2) 
    return norm

In [10]:
def DIMSUMMApper(A, ind, gamma=0.5, nb_it = 1000):
    """
    Efficient Mapper
    input:
    A: sparse matrix
    ind: index of the row that we wish to compute (ri is the i-th row of the matrix, so ind = i)
    gamma: hyperparameter
    nb_it: number of iterations 
    output:
    mean of all the matrices computed
    """
    results = []
    it = 0
    while it<nb_it:
        it+=1
        (m,n) = A.shape 
        ri = A[ind]
        Anorm = norm(A) # norm of columns of A
        AProd = np.zeros((n,n))
        for ci in range(n):
            for cj in range(n):
                random_probs = random.random()  # generate random probabilities
                prob =  min(1,gamma/(Anorm[ci] * Anorm[cj]))
                if prob>random_probs:
                    AProd[ci,cj] += ri[ci]*ri[cj]
                    
        results.append(AProd)
    
    results = np.array(results)
    return results.sum(axis=0)/(len(results))

In [11]:
A = sparse_generator(4,3, 1)

Comparing the Naive and the efficient versions for the first row of matrix A:

In [12]:
DIMSUMMApper(A, 0)

C:\Users\biafa\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in double_scalars


array([[0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.06774211]])

In [13]:
NaiveMapper(A[0])

matrix([[ 0.        ,  0.        , -0.        ],
        [ 0.        ,  0.        , -0.        ],
        [-0.        , -0.        ,  0.13602833]])

#### 4) Algorithm 4: DIMSUMReducer

In [14]:
def DIMSUMReducer(A, col_maps, gamma = 0.5):
    """
    Efficient Reducer
    input:
    A: sparse matrix
    col_maps: list of matrices from the mapper function (for each row of A)
    gamma: hyperparameter
    output:
    result_mat: matrix of size (n,n)
    """
    (nrow, ncol) = col_maps[0].shape
    result_mat = np.zeros((nrow, ncol) )
    Anorm = norm(A) 
    result_mat = sum(col_maps)
    #for lm in col_maps:
        #result_mat+=np.matrix(lm)
    for i in range(nrow):
        for j in range(ncol):
            if gamma> Anorm[i]*Anorm[j]:
                result_mat[i][j]=1/(Anorm[i]*Anorm[j]) * result_mat[i][j]
            else:
                result_mat[i][j]= (1/gamma) * result_mat[i][j]
    return result_mat

In [15]:
def DIMSUMProd(A, gamma = 0.5, nb_it = 1500):
    """
    Function that efficiently computes A^T A for sparse matrix
    input:
    A: sparse matrix
    gamma: hyperparameter
    nb_it: number of iterations for the mapper function
    output:
    prod: matrix equal to A^T A
    """
    (m,n) = A.shape
    col_maps = [DIMSUMMApper(A,j, gamma, nb_it) for j in range(m)]
    prod = DIMSUMReducer(A, col_maps)
    prod[np.isnan(prod)] = 0
    return prod

Comparing the results for the 3 methods (numpy function, naive and efficient version):

In [16]:
A.T@A

array([[0.        , 0.        , 0.        ],
       [0.        , 0.97702171, 0.        ],
       [0.        , 0.        , 1.0000915 ]])

In [17]:
DIMSUMProd(A)

C:\Users\biafa\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\biafa\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\biafa\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars


array([[0.        , 0.        , 0.        ],
       [0.        , 0.98483788, 0.        ],
       [0.        , 0.        , 0.94938266]])

In [18]:
NaiveProd(A)

array([[0.        , 0.        , 0.        ],
       [0.        , 0.97702171, 0.        ],
       [0.        , 0.        , 1.0000915 ]])